In [1]:
#TODO filter years so it'll match the years in the other dataset (awards mostly)
#TODO manage the missing values and NaNs
#TODO manage the outliers

In [2]:
import pandas as pd

#open the csv file of actors
actors = pd.read_csv('actors.csv')

#open the csv file of awards
awards = pd.read_csv('data_csv_awards.csv')

#open the movie tsv file
movies = pd.read_csv('movie_summaries_ada/movie.metadata.tsv', sep='\t', header=None)
#define the columns
movies.columns = ['wikipedia_id', 'freebase_id', 'name', 'release_date', 'revenue', 'runtime', 'languages', 'countries', 'genres']

#open the character.metadata.tsv file
characters = pd.read_csv('movie_summaries_ada/character.metadata.tsv', sep='\t', header=None)
#define the columns of the character file
characters.columns = ['wikipedia_movie_id','freebase_movie_id','movie_release_date','character_name','actor_birth',
                      'actor_gender','actor_height','actor_etnicity','actor_name','actor_age_at_release','freebase_char_actor_map_id','freebase_character_id','freebase_actor_id']

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [3]:
#load the heavy files aka titles imbd to find the imdb unique id, then use it to find the imdb rating and merge it with the movies dataframe
titles_imbd = pd.read_csv('IMDB/title.akas.tsv', sep='\t')

# where only keep the region as US or GB, only the ones that could have an impact in the US

#filter the titles_imbd dataframe to only keep the rows with the region as US or GB
titles_imbd = titles_imbd[(titles_imbd.region == 'US') | (titles_imbd.region == 'GB')]
#we will keep multiple instances of same movie and average the ratings, we'll manage the duplicates in the movies dataframe
#only keep the columns that we need
titles_imbd = titles_imbd[['titleId', 'title', 'region']]

print(titles_imbd.head())

/var/folders/nb/6_kh75dx3xb3cgzlclgmvcx00000gn/T/ipykernel_5573/3284836425.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  titles_imbd = pd.read_csv('IMDB/title.akas.tsv', sep='\t')


      titleId                   title region
5   tt0000001              Carmencita     US
14  tt0000002  The Clown and His Dogs     US
22  tt0000003            Poor Pierrot     GB
33  tt0000005        Blacksmith Scene     US
35  tt0000005  The Blacksmith's Forge     GB


In [4]:
#add the titleId column to the movies dataframe if the title is the same
movies = movies.merge(titles_imbd, left_on='name', right_on='title', how='left')
#drop the title and region columns
movies = movies.drop(columns=['title', 'region'])
#drop the duplicates of titleId (we don't want multiple times the same imdb rating, but we want all ratings of same movie so we remove the duplicates on id but not on name)
movies = movies.drop_duplicates(subset=['titleId'], keep='first')
#only movie that the merge fails
movies.iloc[1,9] = 'tt0245916'
#count the number of movies that have a titleId
print(str(movies.titleId.count())+' movies have a titleId over '+str(movies.name.count())+' movies, so '+str(round(movies.titleId.count()*100/movies.name.count()))+' % movies do have a titleId')
#multiple ratings (titleId here) per movie, how to handle that? average with the number of votes as weight

208354 movies have a titleId over 208354 movies, so 100 % movies do have a titleId


In [5]:
#load the imdb rating file
rating_imbd = pd.read_csv('IMDB/title.ratings.tsv', sep='\t')

In [7]:
movies.head()

,wikipedia_id,freebase_id,name,release_date,revenue,runtime,languages,countries,genres,titleId
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",tt0228333
2,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp...",tt0245916
5,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}",tt0083949
6,13696889,/m/03cfc81,The Gangsters,1913-05-29,NaN,35.0,"{""/m/06ppq"": ""Silent film"", ""/m/02h40lc"": ""Eng...","{""/m/09c7w0"": ""United States of America""}","{""/m/02hmvc"": ""Short Film"", ""/m/06ppq"": ""Silen...",tt0002894
7,13696889,/m/03cfc81,The Gangsters,1913-05-29,NaN,35.0,"{""/m/06ppq"": ""Silent film"", ""/m/02h40lc"": ""Eng...","{""/m/09c7w0"": ""United States of America""}","{""/m/02hmvc"": ""Short Film"", ""/m/06ppq"": ""Silen...",tt0004002


In [9]:
movies[movies.name == 'The Gangsters']

,wikipedia_id,freebase_id,name,release_date,revenue,runtime,languages,countries,genres,titleId
6,13696889,/m/03cfc81,The Gangsters,1913-05-29,NaN,35.0,"{""/m/06ppq"": ""Silent film"", ""/m/02h40lc"": ""Eng...","{""/m/09c7w0"": ""United States of America""}","{""/m/02hmvc"": ""Short Film"", ""/m/06ppq"": ""Silen...",tt0002894
7,13696889,/m/03cfc81,The Gangsters,1913-05-29,NaN,35.0,"{""/m/06ppq"": ""Silent film"", ""/m/02h40lc"": ""Eng...","{""/m/09c7w0"": ""United States of America""}","{""/m/02hmvc"": ""Short Film"", ""/m/06ppq"": ""Silen...",tt0004002
8,13696889,/m/03cfc81,The Gangsters,1913-05-29,NaN,35.0,"{""/m/06ppq"": ""Silent film"", ""/m/02h40lc"": ""Eng...","{""/m/09c7w0"": ""United States of America""}","{""/m/02hmvc"": ""Short Film"", ""/m/06ppq"": ""Silen...",tt0139667


In [10]:
#merge the rating_imbd dataframe with the movies dataframe and keep the average rating for each movie (weighted by the number of votes)
movies_test = movies.merge(rating_imbd, left_on='titleId', right_on='tconst', how='left')
#drop the tconst column
movies_test = movies_test.drop(columns=['tconst'])
#group by name and average the rating
movies_test = movies_test.groupby('name',as_index=False).apply(lambda group: (group['averageRating'] * group['numVotes']).sum() / group['numVotes'].sum())
#TODO:average the rating per name with the number of votes as weight

movies_test.head()

/var/folders/nb/6_kh75dx3xb3cgzlclgmvcx00000gn/T/ipykernel_5573/1964931804.py:6: RuntimeWarning: invalid value encountered in scalar divide
  movies_test = movies_test.groupby('name',as_index=False).apply(lambda group: (group['averageRating'] * group['numVotes']).sum() / group['numVotes'].sum())


,name,None
0,!Women Art Revolution,6.800000
1,#1 Cheerleader Camp,3.600000
2,$,6.300000
3,$9.99,6.703173
4,'68,5.900000


In [21]:
movies_test.shape

(48268, 2)

In [13]:
#merge the movies dataframe with the movies_test dataframe to add the average rating column
movies = movies.merge(movies_test, left_on='name', right_on='name', how='left')
#rename the column
movies = movies.rename(columns={None:'averageRating'})
#drop the duplicate name column
movies = movies.drop(columns=['titleId'])
#drop the duplicates in the name column
movies = movies.drop_duplicates(subset=['name'], keep='first')
movies.head()

Index(['wikipedia_id',  'freebase_id',         'name', 'release_date',
            'revenue',      'runtime',    'languages',    'countries',
             'genres',      'titleId',           None],
      dtype='object')

In [19]:
print(str(movies.averageRating.count())+' movies have a rating over '+str(movies.name.count())+' movies, so '+str(round(movies.averageRating.count()*100/movies.name.count()))+' % movies do have a rating')

45072 movies have a rating over 48268 movies, so 93 % movies do have a rating


In [22]:
#filter the awards dataframe to only keep the rows with winner as True
#awards_winners = awards[awards.winner == True]

#filter the awards dataframe to only keep the rows with category containing ACTOR or ACTRESS
awards_actors = awards[awards.category.str.contains('ACTOR|ACTRESS')]
#awards_winners[awards_winners.category.str.contains('ACTOR|ACTRESS')]

In [23]:
#count the number of awards per entity in awards_actors
awards_actors_count = awards_actors.groupby('entity',as_index=False).count()

In [24]:
# get a list of unique actors from the character dataframe
actors_list = pd.DataFrame(characters.actor_name.unique())
print(actors_list.shape)

#rename the column
actors_list.columns = ['name']

#add a column to the actors_list dataframe with the winners count
actors_list = actors_list.merge(awards_actors_count[['entity','winner']], left_on='name', right_on='entity', how='left')
#drop the entity column
actors_list = actors_list.drop(columns=['entity'])
#rename the count column
actors_list = actors_list.rename(columns={'winner': 'awards'})
#fill the NaN values in the awards column with 0
actors_list['awards'] = actors_list['awards'].fillna(0)
print(actors_list.shape)

#create a dataframe with the number of movies per actor
actors_movies_count = pd.DataFrame(characters.actor_name.value_counts())

#add a column to the actors_list dataframe with the movies count
actors_list = actors_list.merge(actors_movies_count, left_on='name', right_index=True, how='left')
#rename the count column
actors_list = actors_list.rename(columns={'actor_name': 'movies'})
print(actors_list.shape)

#replace the Actor column of actors dataframe by the same value but replace each _ by a space
actors['Actor'] = actors['Actor'].str.replace('_', ' ')

#add the actors dataframe to the actors_list dataframe on the Actor column and name column respectively
actors_list = actors_list.merge(actors, left_on='name', right_on='Actor', how='left')

#drop the Actor column
actors_list = actors_list.drop(columns=['Actor'])
print(actors_list.shape)
#print the number of actors with a Fame value
print(str(actors_list.Fame.count())+' actors have a Fame value over '+str(actors_list.name.count())+' actors, so '+str(round(actors_list.Fame.count()*100/actors_list.name.count()))+' % actors do have a Fame value')
#replace the NaN values in the Fame column by 0
actors_list['Fame'] = actors_list['Fame'].fillna(0)
#replace the NaN values in the Liked,Disliked,Neutral columns by 0
actors_list['Liked'] = actors_list['Liked'].fillna(0)
actors_list['Disliked'] = actors_list['Disliked'].fillna(0)
actors_list['Neutral'] = actors_list['Neutral'].fillna(0)

#TODO:issue to solve, we loose about 46 actors in the merge because the name has . or ' or -, and we replace every _ by a space
#TODO: possible solution, replace every . or ' or - by a space in the actors_list dataframe
#TODO: possible solution, replace every . or ' or - or space by a _ in the actors dataframe

#TODO: NaN values to manage

#TODO: add gender and ethnicity to the actors_list dataframe

(134079, 1)
(134079, 2)
(134079, 3)
(134079, 7)
1310 actors have a Fame value over 134078 actors, so 1 % actors do have a Fame value


In [25]:
#make a pandas dataframe of the characters dataframe with only the first row of each actor
actors_metadata = characters.drop_duplicates(subset=['actor_name'], keep='first')
#drop the columns that we don't need
actors_metadata = actors_metadata.drop(columns=['wikipedia_movie_id', 'freebase_movie_id', 'movie_release_date', 'character_name','actor_age_at_release', 'freebase_char_actor_map_id', 'freebase_character_id'])
actors_metadata.head()

,actor_birth,actor_gender,actor_height,actor_etnicity,actor_name,freebase_actor_id
0,1958-08-26,F,1.620,NaN,Wanda De Jesus,/m/03wcfv7
1,1974-08-15,F,1.780,/m/044038p,Natasha Henstridge,/m/0346l4
2,1969-06-15,M,1.727,/m/0x67,Ice Cube,/m/01vw26l
3,1967-09-12,M,1.750,NaN,Jason Statham,/m/034hyc
4,1977-09-25,F,1.650,NaN,Clea DuVall,/m/01y9xg


In [1]:
#TODO: create dataframe with awards until each year per actor, so we can easily add the awards to the movies dataframe and characters dataframe

#Use a csr sparse matrix to store the awards per actor (row index) and sum the values until given year (column index) for each call

In [26]:
print(actors_list.shape)
actors_list.head()

(134079, 7)


,name,awards,movies,Fame,Liked,Disliked,Neutral
0,Wanda De Jesus,0.0,8.0,0.0,0.0,0.0,0.0
1,Natasha Henstridge,0.0,23.0,0.0,0.0,0.0,0.0
2,Ice Cube,0.0,33.0,93.0,57.0,11.0,25.0
3,Jason Statham,0.0,31.0,78.0,59.0,4.0,14.0
4,Clea DuVall,0.0,31.0,0.0,0.0,0.0,0.0


In [27]:
print(movies.shape)
movies.head()
#TODO: add number of actor awards per movie, number of actors per movie, 
# average number of movies per actor (aka experience), average fame per movie (aka popularity)
#TODO: (idk how yet) number of awards (actor and non actor) per movie

(48268, 10)


,wikipedia_id,freebase_id,name,release_date,revenue,runtime,languages,countries,genres,averageRating
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",4.900000
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp...",6.000000
2,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}",5.900000
3,13696889,/m/03cfc81,The Gangsters,1913-05-29,NaN,35.0,"{""/m/06ppq"": ""Silent film"", ""/m/02h40lc"": ""Eng...","{""/m/09c7w0"": ""United States of America""}","{""/m/02hmvc"": ""Short Film"", ""/m/06ppq"": ""Silen...",6.043617
6,18998739,/m/04jcqvw,The Sorcerer's Apprentice,2002,NaN,86.0,"{""/m/02h40lc"": ""English Language""}","{""/m/0hzlz"": ""South Africa""}","{""/m/0hqxf"": ""Family Film"", ""/m/01hmnh"": ""Fant...",6.108016


In [9]:
# basic info of the data to see changes when cleaning
print(actors.info())
print(awards.info())
print(movies.info())
print(characters.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1355 entries, 0 to 1354
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Actor     1355 non-null   object
 1   Fame      1355 non-null   int64 
 2   Liked     1355 non-null   int64 
 3   Disliked  1355 non-null   int64 
 4   Neutral   1355 non-null   int64 
dtypes: int64(4), object(1)
memory usage: 53.1+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11058 entries, 0 to 11057
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   year      11058 non-null  int64 
 1   category  11058 non-null  object
 2   winner    11058 non-null  bool  
 3   entity    11058 non-null  object
dtypes: bool(1), int64(1), object(2)
memory usage: 270.1+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81741 entries, 0 to 81740
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        